In [72]:
import pandas as pd
import os, json

In [73]:
COLUMNS_ORDER = ["Ano","Tipo","Instituição","IBGE","UF","População","Coluna","Conta","Descrição Conta","Identificador da Conta","Valor"]
YEARS = ["2022","2021","2020","2019","2018","2017","2016","2015","2014","2013"]
DATA_PATH = "data"
FILES_NAMES = os.listdir(os.path.join(DATA_PATH,"2022"))

with open("file_field.json", "r", encoding="latin-1") as f:
    file_field = json.load(f)

In [74]:
to_float = lambda x: x.replace(",", ".")

def get_conta(x:str) -> str:
    sx = x.split(" - ")
    if len(sx) == 1:
        return "Sem Numero"
    return sx[0]

def get_des_conta(x:str) -> str:
    sx = x.split(" - ")
    if len(sx) == 1:
        return x
    return sx[1]

In [75]:
def read_and_format_df(year:str, file_name:str) -> pd.DataFrame:
    file_path = os.path.join(DATA_PATH,year,file_name)
    df = pd.read_csv(file_path, sep=";", encoding="latin-1", skiprows=3)
    df["Valor"] = df["Valor"].apply(to_float)
    df["Valor"] = df["Valor"].astype(float)
    df["Descrição Conta"] = df["Conta"].apply(get_des_conta)
    df["Conta"] = df["Conta"].apply(get_conta)
    df["Ano"] = int(year)
    df["Tipo"] = file_field[file_name]
    df = df.rename(columns={"Cod.IBGE":"IBGE"})
    return df[COLUMNS_ORDER]

In [76]:
balanco_patrimonial = pd.concat([read_and_format_df(year,FILES_NAMES[0]) for year in [YEARS[0]]])
depesas_orcamentarias = pd.concat([read_and_format_df(year,FILES_NAMES[1]) for year in [YEARS[0]]])
despesas_por_funcao = pd.concat([read_and_format_df(year,FILES_NAMES[2]) for year in [YEARS[0]]])
execucao_restos_pagar = pd.concat([read_and_format_df(year,FILES_NAMES[3]) for year in [YEARS[0]]])
execucao_restos_pagar_funcao = pd.concat([read_and_format_df(year,FILES_NAMES[4]) for year in [YEARS[0]]])
receitas_orcamentarias = pd.concat([read_and_format_df(year,FILES_NAMES[5]) for year in [YEARS[0]]])
variacoes_patrimoniais = pd.concat([read_and_format_df(year,FILES_NAMES[6]) for year in [YEARS[0]]])

In [77]:
full_df = pd.concat([balanco_patrimonial,
                     depesas_orcamentarias,
                     despesas_por_funcao,
                     execucao_restos_pagar,
                     execucao_restos_pagar_funcao,
                     receitas_orcamentarias,
                     variacoes_patrimoniais])

In [78]:
full_df.tail()

,Ano,Tipo,Instituição,IBGE,UF,População,Coluna,Conta,Descrição Conta,Identificador da Conta,Valor
727919,2022,VariaÃ§Ãµes Patrimoniais,Prefeitura Municipal de Delta - MG,3121258,MG,10994,31/12/2022,4.9.9.6.0.00.00,Indenizações,siconfi-cor_P4.9.9.6.0.00.00,92957.12
727920,2022,VariaÃ§Ãµes Patrimoniais,Prefeitura Municipal de Delta - MG,3121258,MG,10994,31/12/2022,4.9.9.6.1.00.00,Indenizações,siconfi-cor_P4.9.9.6.1.00.00,92957.12
727921,2022,VariaÃ§Ãµes Patrimoniais,Prefeitura Municipal de Delta - MG,3121258,MG,10994,31/12/2022,4.9.9.9.0.00.00,Variações Patrimoniais Aumentativas Decorrente...,siconfi-cor_P4.9.9.9.0.00.00,151859.17
727922,2022,VariaÃ§Ãµes Patrimoniais,Prefeitura Municipal de Delta - MG,3121258,MG,10994,31/12/2022,4.9.9.9.1.00.00,Variações Patrimoniais Aumentativas Decorrente...,siconfi-cor_P4.9.9.9.1.00.00,151859.17
727923,2022,VariaÃ§Ãµes Patrimoniais,Prefeitura Municipal de Delta - MG,3121258,MG,10994,31/12/2022,Sem Numero,Resultado Patrimonial do Período,siconfi-cor_ResultadoPatrimonialDoPeriodo,12502832.90


In [79]:
full_df.dtypes

Ano                         int64
Tipo                       object
Instituição                object
IBGE                        int64
UF                         object
População                   int64
Coluna                     object
Conta                      object
Descrição Conta            object
Identificador da Conta     object
Valor                     float64
dtype: object

In [80]:
full_df["Tipo"].str.len().max()

41